In [2]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/D1/trainX.npy')
y_train = np.load('gdrive/My Drive/D1/trainy.npy')
x_test = np.load('gdrive/My Drive/D1/testX.npy')
y_test = np.load('gdrive/My Drive/D1/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(33104, 128, 6) (33104, 118) (3740, 128, 6) (3740, 118)


In [3]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from tensorflow import keras
from pandas import read_csv
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
# from keras.utils import to_categorical
from matplotlib import pyplot
from keras import regularizers
import tensorflow as tfs

In [4]:
# reshape data into time steps of sub-sequences
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
n_steps, n_length = 4, 32
x_train = x_train.reshape((x_train.shape[0], n_steps, n_length, n_features))
x_test = x_test.reshape((x_test.shape[0], n_steps, n_length, n_features))

In [5]:
from sklearn.model_selection import train_test_split
X_training, X_validation, y_training, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [6]:
model2 = Sequential()
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu'), input_shape=(None,n_length,n_features)))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=5, activation='relu')))
model2.add(TimeDistributed(Dropout(0.5)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model2.add(TimeDistributed(Flatten()))
model2.add(LSTM(100))
model2.add(Dropout(0.5))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(n_outputs, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()
callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, None, 28, 64)     1984      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, None, 24, 64)     20544     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, None, 20, 64)     20544     
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, None, 16, 64)     20544     
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, None, 16, 64)     0

In [7]:
# fit network
history = model2.fit(X_training, y_training, epochs=200, verbose=True, validation_data=(X_validation, y_validation), batch_size=128)

Epoch 1/200
207/207 [==============================] - 46s 202ms/step - loss: 3.0955 - accuracy: 0.2199 - val_loss: 1.7564 - val_accuracy: 0.5264
Epoch 2/200
207/207 [==============================] - 42s 203ms/step - loss: 1.4998 - accuracy: 0.5775 - val_loss: 0.8630 - val_accuracy: 0.7608
Epoch 3/200
207/207 [==============================] - 43s 210ms/step - loss: 0.9694 - accuracy: 0.7344 - val_loss: 0.5477 - val_accuracy: 0.8565
Epoch 4/200
207/207 [==============================] - 42s 201ms/step - loss: 0.7248 - accuracy: 0.8033 - val_loss: 0.4436 - val_accuracy: 0.8817
Epoch 5/200
207/207 [==============================] - 42s 202ms/step - loss: 0.6067 - accuracy: 0.8311 - val_loss: 0.3771 - val_accuracy: 0.8888
Epoch 6/200
207/207 [==============================] - 43s 206ms/step - loss: 0.5136 - accuracy: 0.8576 - val_loss: 0.3870 - val_accuracy: 0.8872
Epoch 7/200
207/207 [==============================] - 42s 202ms/step - loss: 0.4707 - accuracy: 0.8643 - val_loss: 0.3162 -

In [8]:
# evaluate model
loss, accuracy = model2.evaluate(X_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))

Validation Accuracy:  0.9686
Testing Accuracy: 0.8898
